In [2]:
#%matplotlib inline
import argparse
import os
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.utils.data import Dataset,DataLoader
import torchvision.transforms as transforms
import torchvision.utils as vutils
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML
import pandas as pd
import h5py
from PIL import Image
from io import BytesIO
import tqdm

# Set random seed for reproducibility
manualSeed = 999
#manualSeed = random.randint(1, 10000) # use if you want new results
print("Random Seed: ", manualSeed)
random.seed(manualSeed)
torch.manual_seed(manualSeed)
torch.use_deterministic_algorithms(False) # Needed for reproducible results

/data1/home/nidhi2023/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Random Seed:  999


In [3]:
# !kaggle competitions download -c isic-2024-challenge

In [4]:
# Setup device-agnostic code
device = "cuda:1" if torch.cuda.is_available() else "cpu"
device

'cuda:1'

In [5]:
train_matadata = pd.read_csv("../../wgangp/train-metadata.csv", low_memory=False)
train_matadata.head()

,isic_id,target,patient_id,age_approx,sex,anatom_site_general,clin_size_long_diam_mm,image_type,tbp_tile_type,tbp_lv_A,...,lesion_id,iddx_full,iddx_1,iddx_2,iddx_3,iddx_4,iddx_5,mel_mitotic_index,mel_thick_mm,tbp_lv_dnn_lesion_confidence
0,ISIC_0015670,0,IP_1235828,60.0,male,lower extremity,3.04,TBP tile: close-up,3D: white,20.244422,...,NaN,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,97.517282
1,ISIC_0015845,0,IP_8170065,60.0,male,head/neck,1.10,TBP tile: close-up,3D: white,31.712570,...,IL_6727506,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,3.141455
2,ISIC_0015864,0,IP_6724798,60.0,male,posterior torso,3.40,TBP tile: close-up,3D: XP,22.575830,...,NaN,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,99.804040
3,ISIC_0015902,0,IP_4111386,65.0,male,anterior torso,3.22,TBP tile: close-up,3D: XP,14.242329,...,NaN,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,99.989998
4,ISIC_0024200,0,IP_8313778,55.0,male,anterior torso,2.73,TBP tile: close-up,3D: white,24.725520,...,NaN,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,70.442510


In [6]:
class ImageLoader(Dataset):
    def __init__(self, df, file_hdf, transform=None):
        self.df = df
        self.fp_hdf = h5py.File(file_hdf, mode="r")
        # self.isic_ids = df[df['target']!=1]['isic_id'].values
        self.isic_ids = df['isic_id'].values
#         self.targets = df['target'].values
        self.transform = transform
        
    def __len__(self):
        return len(self.isic_ids)
    
    def __getitem__(self, index):
        isic_id = self.isic_ids[index]
        # print(isic_id)
        image = Image.open(BytesIO(self.fp_hdf[isic_id][()]))
#         target = self.targets[index]
#         image = torch.permute(image,(2,1,0))
        if self.transform:
            return (self.transform(image))
        else:
            return (image)

In [9]:
train_transforms = transforms.Compose([
                               transforms.Resize(128),
                               transforms.CenterCrop(128),
                               transforms.ToTensor(),
                               transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                           ])

train_dataset_full = ImageLoader(train_matadata,
                      file_hdf = "../../wgangp/train-image.hdf5",
                      transform=train_transforms
                     )


# train_size = int(0.7 * len(train_dataset_full))
# test_size = len(train_dataset_full) - train_size
# train_dataset, test_dataset = torch.utils.data.random_split(train_dataset_full, [train_size, test_size])

In [10]:
len(train_dataset_full)

401059

In [11]:
dataloader = torch.utils.data.DataLoader(train_dataset_full, batch_size=128,num_workers=8,shuffle=True)

# Decide which device we want to run on
device = torch.device("cuda" if (torch.cuda.is_available()) else "cpu")

ds = iter(dataloader)


# next(ds).shape

In [12]:
# Define the autoencoder model
class Autoencoder(nn.Module):
    def __init__(self,latent_dim):
        super(Autoencoder, self).__init__()

        self.encoder = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, stride=2, padding=1),  # (3, 156, 156) -> (64, 78, 78)
            nn.ReLU(True),
            nn.Conv2d(16, 32, kernel_size=3, stride=2, padding=1),  # (64, 78, 78) -> (128, 39, 39)
            nn.ReLU(True),
            nn.Conv2d(32, 32, kernel_size=3, stride=2, padding=1),  # (128, 39, 39) -> (256, 20, 20)
            nn.ReLU(True),
            nn.Conv2d(32, 64, kernel_size=3, stride=2, padding=1),  # (128, 39, 39) -> (256, 20, 20)
            nn.ReLU(True),
            nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1),  # (128, 39, 39) -> (256, 20, 20)
            nn.ReLU(True),
            nn.Flatten(),  # Image grid to single feature vector
            nn.Linear(128 * 16 , latent_dim),
        )
        self.linear = nn.Sequential(nn.Linear(latent_dim, 128 * 16 ), nn.ReLU(True))
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(128, 64, kernel_size=3, stride=2, padding=1, output_padding=1),  # (256, 20, 20) -> (128, 39, 39)
            nn.ReLU(True),
            nn.ConvTranspose2d(64, 32, kernel_size=3, stride=2, padding=1, output_padding=1),  # (128, 39, 39) -> (64, 78, 78)
            nn.ReLU(True),
            nn.ConvTranspose2d(32, 32, kernel_size=3, stride=2, padding=1, output_padding=1),  # (64, 78, 78) -> (3, 156, 156)
            nn.ReLU(True),
            nn.ConvTranspose2d(32, 16, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.ReLU(True),
            nn.ConvTranspose2d(16, 3, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.Tanh()
        )

    def forward(self, x):
        x = self.encoder(x)
        # print(x.shape)
        # y = self.linear(x)
        # y = y.reshape(y.shape[0], -1, 4, 4)
        # y = self.decoder(y)
        return x

In [13]:
# Hyperparameters
num_epochs = 100
learning_rate = 1e-3

# Initialize the model, loss function, and optimizer
model = Autoencoder(latent_dim=256).cuda()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


In [ ]:

max_loss = 9999
# Training loop
for epoch in range(num_epochs):
    avg_loss = 0
    for img in tqdm.tqdm(dataloader):
        img = img.cuda()
        
        # Forward pass
        output = model(img)
        loss = criterion(output, img)
        
        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        avg_loss += loss.item() / len(dataloader)
    output = output.mul(0.5).add(0.5)
    img = img.mul(0.5).add(0.5)
    vutils.save_image(output,'./samples/samples_{}.jpg'.format(epoch))
    vutils.save_image(img,'./samples/input_{}.jpg'.format(epoch))
    if avg_loss < max_loss :
        torch.save(model.state_dict(), 'min_loss_autoencoder.pth')
        max_loss = avg_loss
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}')

# Save the trained model
torch.save(model.state_dict(), 'autoencoder.pth')

print("Training complete!")
